# 3.3 线性回归的简洁实现

随着深度学习框架的发展，开发深度学习应用变得越来越便利。实践中，我们通常可以用比上一节更简洁的代码来实现同样的模型。在本节中，我们将介绍如何使用PyTorch更方便地实现线性回归的训练。

In [2]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.3.0+cpu


## 3.3.1 生成数据集

我们生成与上一节中相同的数据集。其中`features`是训练数据特征，`labels`是标签。

In [3]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [3]:
print(features.size())
print(features[0:10, :])
print(labels.size())
print(labels[0:10])

torch.Size([1000, 2])
tensor([[-0.2366,  0.1843],
        [-0.3527, -0.6695],
        [ 1.0460, -1.1982],
        [-0.1275, -0.3546],
        [-1.4601,  1.0341],
        [-0.3041,  1.0790],
        [-0.6087,  0.8466],
        [ 2.0946,  0.5904],
        [-2.2064, -0.4890],
        [-1.4322, -0.6620]])
torch.Size([1000])
tensor([ 3.0891,  5.8000, 10.3718,  5.1483, -2.2304, -0.0786,  0.1260,  6.3749,
         1.4493,  3.5781])


## 3.3.2 读取数据

PyTorch提供了[`data`](https://pytorch.org/docs/stable/data.html#module-torch.utils.data)包来读取数据。由于`data`常用作变量名，我们将导入的`data`模块用`Data`代替。在每一次迭代中，我们将随机读取包含10个数据样本的小批量。

In [4]:
import torch.utils.data as Data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)

# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)

这里`data_iter`的使用跟上一节中的一样。让我们读取并打印第一个小批量数据样本。

In [5]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[-2.3686, -0.7498],
        [ 0.0485,  0.0653],
        [-2.6128, -0.5734],
        [-0.1371,  0.0523],
        [-0.1684,  1.2629],
        [ 0.1822,  0.0911],
        [ 0.5623,  0.5211],
        [-0.0454, -1.6457],
        [ 0.0725, -0.6547],
        [-0.0364,  0.1855]]) 
 tensor([ 2.0098,  4.0703,  0.9319,  3.7581, -0.4290,  4.2587,  3.5348,  9.7056,
         6.5856,  3.5105])


## 3.3.3 定义模型

在上一节从零开始的实现中，我们需要定义模型参数，并使用它们一步步描述模型是怎样计算的。当模型结构变得更复杂时，这些步骤将变得更繁琐。其实，PyTorch提供了大量预定义的层，这使我们只需关注使用哪些层来构造模型。下面将介绍如何使用PyTorch更简洁地定义线性回归。

首先，导入`torch.nn`模块。实际上，“nn”是neural networks（神经网络）的缩写。顾名思义，该模块定义了大量神经网络的**层**（layer）。`nn`的核心数据结构是`Module`，它是一个抽象概念，既可以表示神经网络中的某个层，也可以表示一个包含很多层的神经网络。在实际使用中，最常见的做法是继承`nn.Module`，撰写自己的网络/层。一个`nn.Module`实例应该包含一些层以及返回输出的前向传播（forward）方法。下面先来看看如何用`nn.Module`实现一个线性回归模型。

In [6]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__() # 5.1.2里面有`super`的详细解释
        self.linear = nn.Linear(n_feature, 1)

    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


`LinearNet`继承自`nn.Module`，其构造函数则调用了`nn.Linear`来定义了自己的linear。[`torch.nn.Linear(in_features, out_features, bias=True)`](https://pytorch.org/docs/stable/nn.html#linear)是PyTorch提供的线性变换功能，它可以对输入的数据进行线性变换：$y = xA^T + b$.

模型网络参数的初始值遵从如下规则：
* **w**eight：w的初始值为$\mathcal{U}(-\sqrt{k}, \sqrt{k})$，其中 $k = \frac{1}{\text{in_features}}$。此处的in_features的值是`mum_inputs`等于2，$\sqrt{0.5}=0.7071$.函数$\mathcal{U}(-\sqrt{k}, \sqrt{k})$的意思是表示w在$[-0.7071,0.771]$上均匀分布。
* **b**ias：b的初始值与w相同。

查看一下上面定义的网络的参数☟

In [7]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.5431, -0.4366]], requires_grad=True)
Parameter containing:
tensor([-0.3663], requires_grad=True)


☝`torch.nn`的参数-[`torch.nn.Parameter`](https://pytorch.org/docs/stable/nn.html#parameters)，也就是模型，即神经网络的参数，它也是一个tensor。因此，`Parameter`被作为`Tensor`的一个子类。通常当网络架构确定了，那么该网络的参数也会相应的确定，而[`parameters()`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module.parameters)函数则被定义成一个迭代器，用于访问这些参数。

在PyTorch中，[`torch.nn.Sequential()`](https://pytorch.org/docs/stable/nn.html#sequential)实例可以看作是一个串联各个层的容器。在构造模型时，我们在该容器中依次添加层。当给定输入数据时，容器中的每一层将依次计算并将输出作为下一层的输入。`torch.nn.Sequential()`的[小例子](../../../DeepLearningWithPyTorch-A60MinuteBlitz/TestSequential.ipynb)。下面给出用`Sequential()`构造模型网络的三种方法：☟

In [8]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )
print(net)
print(net[0])

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [9]:
# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [10]:
# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


![线性回归是一个单层神经网络](../img/chapter03/3.1_linreg.svg)

----
回顾图3.1中线性回归在神经网络图中的表示。作为一个单层神经网络，线性回归输出层中的神经元和输入层中各个输入完全连接。因此，线性回归的输出层又叫全连接层。在上面的`Sequential`容器中，我们定义的层数为1。

## 3.3.4 初始化模型参数
在使用`net`前，我们需要初始化模型参数，如线性回归模型中的权重w和偏差b。PyTorch在`init`模块中提供了多种参数初始化方法。这里的`init`是`initializer`的缩写形式。我们通过`init.normal_`将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布。偏差会初始化为零。

In [11]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [12]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0142, -0.0161]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 3.3.5 定义损失函数
PyTorch在`nn`模块中提供了各种损失函数，这些损失函数可看作是一种特殊的层，PyTorch也将这些损失函数实现为`nn.Module`的子类。我们现在使用它提供的均方误差损失作为模型的损失函数。

In [13]:
loss = nn.MSELoss()

[torch.nn.MSELoss](https://pytorch.org/docs/stable/nn.html#mseloss)均方误差（mean squared error），在此处做如下处理：☟
$$
\ell(w_1, w_2, b) =\frac{1}{n} \sum_{i=1}^n \ell^{(i)}(w_1, w_2, b) =\frac{1}{n} \sum_{i=1}^n \frac{1}{2}\left(x_1^{(i)} w_1 + x_2^{(i)} w_2 + b - y^{(i)}\right)^2
$$

## 3.3.6 定义优化算法
同样，我们也无须自己实现小批量随机梯度下降算法。`torch.optim`模块提供了很多常用的优化算法比如`SGD`、`Adam`和`RMSProp`等。下面我们创建一个用于优化`net`所有参数的优化器实例，并指定学习率为0.03的小批量随机梯度下降（`SGD`）为优化算法。

In [14]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


[`torch.optim.SGD`](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD)类提供随机梯度下降的优化方法。

我们还可以为不同子网络设置不同的学习率，这在finetune时经常用到。例：☟

In [15]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

有时候我们不想让学习率固定成一个常数，那如何调整学习率呢？主要有两种做法。一种是修改optimizer.param_groups中对应的学习率，另一种是更简单也是较为推荐的做法——新建优化器，由于optimizer十分轻量级，构建开销很小，故而可以构建新的optimizer。但是后者对于使用动量的优化器（如Adam），会丢失动量等状态信息，可能会造成损失函数的收敛出现震荡等情况。☟

In [16]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

## 3.3.7 训练模型
在使用PyTorch训练模型时，我们通过调用`optim`实例的`step`函数来迭代模型参数。按照小批量随机梯度下降的定义，我们在`step`函数中指明批量大小，从而对批量中样本梯度求平均。

In [17]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000393
epoch 2, loss: 0.000034
epoch 3, loss: 0.000061


下面我们分别比较学到的模型参数和真实的模型参数。我们从`net`获得需要的层，并访问其权重（`weight`）和偏差（`bias`）。学到的参数和真实的参数很接近。

In [18]:
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

[2, -3.4] tensor([[ 2.0002, -3.4009]])
4.2 tensor([4.2004])


## 小结

* 使用PyTorch可以更简洁地实现模型。
* `torch.utils.data`模块提供了有关数据处理的工具，`torch.nn`模块定义了大量神经网络的层，`torch.nn.init`模块定义了各种初始化方法，`torch.optim`模块提供了模型参数初始化的各种方法。

## 练习

* 查阅PyTorch文档，看看[`Loss functions`](https://pytorch.org/docs/stable/nn.html#loss-functions)和[`torch.nn.init`](https://pytorch.org/docs/stable/nn.init.html#torch-nn-init)模块里提供了哪些损失函数和初始化方法。
* 如何访问`dense.weight`的梯度？


# 扫码直达[知乎专栏](https://zhuanlan.zhihu.com/unicom-d2l)

![](../img/zhihu.png)